Install necessary packages.
transformers contains the model and many necessary pre-processing tools
peft.git contains components for Parameter efficient fine-tuning
wandb for tracking performance
bitsandbytes and accelerate.git are supports for quantitizing the model into 4-bit

In [ ]:
!pip install -q transformers==4.30 einops
!pip install git+https://github.com/huggingface/accelerate
!pip install transformers[torch]
!pip install git+https://github.com/huggingface/peft.git
!pip install datasets bitsandbytes wandb trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 71.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.1 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-ssgfyw97
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-req-build-ssgfyw97
  Resolved https://github.com/huggingface/accelerate to commit 80da9cfb09bb3cc9f1b385cb55d6b90d025a5fd9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-0.24.0.dev0-py3-none-any.whl size=258139 sha256=bd8a73907827299c4513cf862b13948306f5536588ada49fff20e50

Load the model with 4-bit quantitisation

In [ ]:
import torch
import transformers
from transformers import AutoModelForCausalLM, BitsAndBytesConfig


model_name = "tiiuae/falcon-rw-1b"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant = True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-rw-1b:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-rw-1b:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Load dataset and preprocess it

In [13]:
from datasets import load_dataset
from transformers import AutoTokenizer

dataset = load_dataset("covid_qa_deepset")
dataset = dataset['train'].train_test_split(test_size=606, train_size=1413)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

def convert_answer(row):
  answer = row['answers']['text'][0]
  row['answers'] = answer
  return row

def shorten_context(row):
  #if sequence of context is longer than 512, it will be cut off
  if len(torch.split(row["context"], " ")) > 512:
    input_seq = tokenizer.encode("sumarize: " + row["context"], return_tensors="pt", max_length=512, truncation=True)
    summarized_seq = model.generate(input_seq, min_length = 100, max_length = 512)
    row["context"] = tokenizer.decode(summarized_seq[0])
  return row

def output_text(row):
  row['text'] = "### Context: " + row['context'] +  "### Question: " + row['question'] + "### Answer: " + row['answers']
  return row

train_data = dataset["train"]
train_data.add_column(name="text", column=train_data['answers'])
train_data = train_data.map(convert_answer)
train_data = train_data.map(shorten_context)
train_data = train_data.map(output_text)
train_data.remove_columns('document_id')
train_data.remove_columns('is_impossible')
train_data.remove_columns('id')

Flattening the indices:   0%|          | 0/1413 [00:00<?, ? examples/s]

Map:   0%|          | 0/1413 [00:00<?, ? examples/s]

Map:   0%|          | 0/1413 [00:00<?, ? examples/s]

Map:   0%|          | 0/1413 [00:00<?, ? examples/s]

Dataset({
    features: ['document_id', 'context', 'question', 'is_impossible', 'answers', 'text'],
    num_rows: 1413
})

In [14]:
print(dataset)
print(dataset.column_names)

DatasetDict({
    train: Dataset({
        features: ['document_id', 'context', 'question', 'is_impossible', 'id', 'answers'],
        num_rows: 1413
    })
    test: Dataset({
        features: ['document_id', 'context', 'question', 'is_impossible', 'id', 'answers'],
        num_rows: 606
    })
})
{'train': ['document_id', 'context', 'question', 'is_impossible', 'id', 'answers'], 'test': ['document_id', 'context', 'question', 'is_impossible', 'id', 'answers']}


In [24]:
dataset['train']['answers'][0]['text']

['among older adults, especially males, with pre-existing diseases.']

In [15]:
dataset['train']['answers'][0]['text'][0]

'among older adults, especially males, with pre-existing diseases.'

Configure LoRa finetuning

In [10]:
from peft import LoraConfig

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "query_key_value",
        "dense",
        "dense_h_to_4h",
        "dense_4h_to_h",
    ]
)

Create training arguments

In [11]:
from transformers import TrainingArguments



training_arguments = TrainingArguments(

    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs = 1,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=5,
    max_steps = 500,
    optim = "paged_adamw_32bit",
    output_dir = "./results",
    lr_scheduler_type = "constant",
    warmup_ratio = 0.03,
    max_grad_norm = 0.3,
    save_steps = 10,
    group_by_length=True,
    report_to=None
)

Load the supervise fine-tuning trainer

In [17]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:133: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/1413 [00:00<?, ? examples/s]

In [18]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

Finally, train the model

In [20]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [22]:
import wandb

wandb.init()

wandb: Currently logged in as: bobbyngoson204 (intelligentsystem). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.15.10
wandb: Run data is saved locally in /content/wandb/run-20230921_022106-ql8tuta9
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run twilight-hill-1
wandb: ⭐️ View project at https://wandb.ai/intelligentsystem/uncategorized
wandb: 🚀 View run at https://wandb.ai/intelligentsystem/uncategorized/runs/ql8tuta9


In [23]:
trainer.train()

Step,Training Loss
5,2.184300
10,2.184000
15,2.023800
20,2.032000
25,1.885700
30,1.790400
35,1.655600
40,1.527500
45,1.451800
50,1.241000


KeyboardInterrupt: ignored